In [1]:
import os
import time
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType

In [2]:
# Crear carpeta temporal Spark
temp_path = os.path.join(os.getcwd(), 'spark-temp')
os.makedirs(temp_path, exist_ok=True)

# Definir variables de entorno
os.environ['JAVA_HOME'] = os.environ['CONDA_PREFIX'] + '\Library'
os.environ['SPARK_LOCAL_DIRS'] = temp_path

print('JAVA_HOME:', os.environ.get('JAVA_HOME'))
print('SPARK_LOCAL_DIRS:', os.environ.get('SPARK_LOCAL_DIRS'))

JAVA_HOME: C:\Users\TESTER\anaconda3\envs\btc_portfolio\Library
SPARK_LOCAL_DIRS: C:\Users\TESTER\Desktop\Laboral\GIT\btc-3-asset-portfolio-extension\notebooks\spark-temp


In [3]:
# Crear Spark Session y medir tiempo
start_time = time.time()

spark = SparkSession.builder \
.appName('btcproject') \
.config('spark.driver.memory', '8g') \
.config('spark.executor.memory', '8g') \
.config('spark.local.dir', temp_path) \
.getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

end_time = time.time()
print('Spark Version:', spark.version)
print(f'Tiempo total en crear SparkSession: {round(end_time - start_time, 2)} segundos')

Spark Version: 3.5.4
Tiempo total en crear SparkSession: 13.45 segundos


# Data Cleaning
## Btc Dataset

In [4]:
df_btc = spark.read \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.csv('../data/Bitcoin Historical Data.csv')

In [5]:
df_btc = df_btc.withColumn(
    "month",
    F.date_format(F.to_date("Date", "MM/dd/yyyy"), "yyyy-MM")
)


df_btc = df_btc.withColumn(
    "month",
    F.concat(F.col("month"), F.lit("-01")).cast("date")
).drop("Date")  

df_btc.printSchema()

df_btc.select(
    F.min("month").alias("Fecha mínima"),
    F.max("month").alias("Fecha máxima")
).show()


root
 |-- Price: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Vol.: string (nullable = true)
 |-- Change %: string (nullable = true)
 |-- month: date (nullable = true)

+------------+------------+
|Fecha mínima|Fecha máxima|
+------------+------------+
|  2010-08-01|  2025-04-01|
+------------+------------+



In [ ]:
df_btc.orderBy(F.col("month").asc()).toPandas().head(50)

In [11]:
btc_data = [
    ("2012-11-01", 1000),
    ("2016-07-01", 2000),
    ("2020-05-01", 3000),
    ("2024-04-01", 4000)
]
btc_schema = StructType([
    StructField("month", StringType(), True),
    StructField("price", IntegerType(), True)
])
df_btc = spark.createDataFrame(btc_data, btc_schema)\
              .withColumn("month", F.col("month").cast("date"))
# ------------------------------------------------------------------------

# ----- Creación del DataFrame df_halving -----
halving_data = [
    ("2012-11-01", 1),
    ("2016-07-01", 1),
    ("2020-05-01", 1),
    ("2024-04-01", 1)
]
halving_schema = StructType([
    StructField("month", StringType(), True),
    StructField("halving_flag", IntegerType(), True)
])
df_halving = spark.createDataFrame(halving_data, halving_schema)\
                  .withColumn("month", F.col("month").cast("date"))

# Renombrar la columna 'halving_flag' a 'halving_flag_aux' para evitar duplicados en el join
df_halving = df_halving.withColumnRenamed("halving_flag", "halving_flag_aux")

# Realizar el JOIN
df_joined = df_btc.join(df_halving, on="month", how="left")\
                  .withColumn("halving_flag", F.coalesce(F.col("halving_flag_aux"), F.lit(0)))\
                  .drop("halving_flag_aux")

# Mostrar resultados ordenados por la columna 'month'
df_joined.orderBy("month").show(20)

Py4JJavaError: An error occurred while calling o229.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 14.0 failed 1 times, most recent failure: Lost task 8.0 in stage 14.0 (TID 42) (DESKTOP-1JVP2OE executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheck

In [8]:
df_btc.orderBy(F.col("month").asc()).toPandas().head(10)

Py4JJavaError: An error occurred while calling o104.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 10.0 failed 1 times, most recent failure: Lost task 8.0 in stage 10.0 (TID 16) (DESKTOP-1JVP2OE executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRead

In [ ]:
# Tu código final
spark.stop()
print("SparkSession cerrada correctamente.")
